<a href="https://colab.research.google.com/github/mirpouya/TensorFlow-Tutorial/blob/main/Malaria_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

In [2]:
dataset, dataset_info = tfds.load("malaria", with_info=True, as_supervised=True, shuffle_files=True, split=["train"])

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/27558 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/malaria/1.0.0.incomplete4QPWMW/malaria-train.tfrecord*...:   0%|          …

Dataset malaria downloaded and prepared to /root/tensorflow_datasets/malaria/1.0.0. Subsequent calls will reuse this data.


In [3]:
TRAIN_RATIO = 0.7
VAL_RATIO = 0.2
TEST_RATIO = 0.1

In [4]:
def splits(dataset, TRAIN_RATIO, VAL_RATIO, TEST_RATIO):

  SIZE = len(dataset)

  train_dataset = dataset.take(int(TRAIN_RATIO * SIZE))
  val_dataset = dataset.skip(int(TRAIN_RATIO * SIZE)).take(int(VAL_RATIO * SIZE))
  test_dataset = dataset.skip(int((TRAIN_RATIO + VAL_RATIO) * SIZE))

  return train_dataset, val_dataset, test_dataset

In [5]:
train_dataset, val_dataset, test_dataset = splits(dataset[0], TRAIN_RATIO, VAL_RATIO, TEST_RATIO)

In [6]:
type(train_dataset)

tensorflow.python.data.ops.take_op._TakeDataset

In [7]:
len(train_dataset), len(val_dataset), len(test_dataset)

(19290, 5511, 2756)

Resize and Rescale image

In [8]:
IM_SIZE = 224

def resize_rescale(image, label, image_size = IM_SIZE):
  return tf.image.resize(image, (image_size, image_size)) / 255.0, label

resize and rescale all the datasets

In [9]:
train_dataset = train_dataset.map(resize_rescale)
val_dataset = val_dataset.map(resize_rescale)
test_dataset = test_dataset.map(resize_rescale)

In [10]:
train_dataset, val_dataset, test_dataset

(<_MapDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 <_MapDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 <_MapDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>)

shuffle datasets

In [11]:
BATCH_SIZE = 32

In [12]:
train_dataset = train_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.shuffle(buffer_size=8, reshuffle_each_iteration=True).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [13]:
train_dataset, val_dataset, test_dataset

(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)

Model Creation

<h1> <b> Subclassed model does not work </b> </h2>

In [14]:
from tensorflow.keras.layers import InputLayer, Conv2D, Dense, BatchNormalization, MaxPool2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

In [15]:
from tensorflow.keras.layers import Layer, Input
from tensorflow.keras.models import Model

In [16]:
class FeatureExtractor(Layer):
  def __init__(self, filters=6, kernel_size=3, strides=1, padding="valid", activation="relu", pool_size=2):
    super(FeatureExtractor, self).__init__()

    self.conv2d_1 = Conv2D(filters = filters, kernel_size = kernel_size, strides = strides, padding = padding, activation = activation)
    self.batchnorm_1 = BatchNormalization()
    self.maxpool_1 = MaxPool2D(pool_size = pool_size, strides = strides*2)

    self.conv2d_2 = Conv2D(filters = filters*2, kernel_size = kernel_size, strides = strides, padding = padding, activation = activation)
    self.batchnorm_2 = BatchNormalization()
    self.maxpool_2 = MaxPool2D(pool_size = pool_size, strides = strides*2)

  def call(self, x, training=True):
    x = self.conv2d_1(x)
    x = self.batchnorm_1(x)
    x = self.maxpool_1(x)

    x = self.conv2d_2(x)
    x = self.batchnorm_2(x)
    x = self.maxpool_2(x)

    return x

In [17]:
feature_extract_subclassed = FeatureExtractor(filters=8, kernel_size=3, strides=1, padding="valid", activation="relu", pool_size=2)

In [18]:
class LenetModel(Model):

  def __init__(self):
    super(LenetModel, self).__init__()

    self.feature_extractor = FeatureExtractor(filters=8, kernel_size=3, strides=1, padding="valid", activation="relu", pool_size=2)
    self.flatten = Flatten()
    self.dense_1 = Dense(100, activation="relu")
    self.dense_2 = Dense(10, activation="relu")
    self.dense_3 = Dense(1, activation="sigmoid")
    self.batchnorm_1 = BatchNormalization()
    self.batchnorm_2 = BatchNormalization()

  def call(self, x, training=True):
    x = self.feature_extractor(x)
    x = self.flatten(x)
    x = self.dense_1(x)
    x = self.batchnorm_1(x)
    x = self.dense_2(x)
    x = self.batchnorm_2(x)
    outcome = self.dense_3(x)

In [19]:
lenet_model = LenetModel()

In [20]:
lenet_model(tf.zeros([1, 224, 224, 3]))

In [21]:
lenet_model.compile(
    optimizer = Adam(learning_rate=0.05),
    loss = BinaryCrossentropy(),
    metrics = "Accuracy"
)

In [22]:
# history = lenet_model.fit(train_dataset, validation_data = val_dataset, epochs=3, verbose=1)

<h1> <b> Subclasses model does not work </b> </h1>

In [23]:
LenetModel = tf.keras.Sequential([
    InputLayer(input_shape = (IM_SIZE, IM_SIZE, 3)),

    Conv2D(filters= 6, kernel_size= 5, strides= 1, padding= "valid",activation= "relu"),
    BatchNormalization(),
    MaxPool2D(pool_size= 2, strides= 2),

    Conv2D(filters=16, kernel_size=5, strides=1, padding="valid", activation="relu"),
    BatchNormalization(),
    MaxPool2D(pool_size=2, strides=2),

    Flatten(),

    Dense(100, activation="relu"),
    BatchNormalization(),

    Dense(10, activation="relu"),
    BatchNormalization(),

    Dense(1, activation="sigmoid")
])

LenetModel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 220, 220, 6)       456       
                                                                 
 batch_normalization_6 (Bat  (None, 220, 220, 6)       24        
 chNormalization)                                                
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 110, 110, 6)       0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 106, 106, 16)      2416      
                                                                 
 batch_normalization_7 (Bat  (None, 106, 106, 16)      64        
 chNormalization)                                                
                                                        

In [24]:
LenetModel.compile(optimizer = Adam(learning_rate=0.08),
              loss=BinaryCrossentropy(),
              metrics="accuracy")

In [25]:
history = LenetModel.fit(train_dataset, validation_data = val_dataset, epochs=10, verbose=1)

Epoch 1/10
603/603 [==============================] - 123s 172ms/step - loss: 0.5362 - accuracy: 0.7311 - val_loss: 1.2169 - val_accuracy: 0.5054
Epoch 2/10
603/603 [==============================] - 56s 93ms/step - loss: 0.2684 - accuracy: 0.9135 - val_loss: 0.2516 - val_accuracy: 0.9414
Epoch 3/10
603/603 [==============================] - 50s 82ms/step - loss: 0.2453 - accuracy: 0.9210 - val_loss: 0.2212 - val_accuracy: 0.9376
Epoch 4/10
603/603 [==============================] - 54s 90ms/step - loss: 0.2208 - accuracy: 0.9326 - val_loss: 0.1968 - val_accuracy: 0.9408
Epoch 5/10
603/603 [==============================] - 51s 84ms/step - loss: 0.2070 - accuracy: 0.9370 - val_loss: 0.2734 - val_accuracy: 0.9276
Epoch 6/10
603/603 [==============================] - 52s 86ms/step - loss: 0.1916 - accuracy: 0.9407 - val_loss: 0.1697 - val_accuracy: 0.9447
Epoch 7/10
603/603 [==============================] - 52s 85ms/step - loss: 0.1762 - accuracy: 0.9436 - val_loss: 0.2118 - val_accurac

<h2> <b> Save and Load the model </b> </h2>

In [42]:
LenetModel.save("lenet")

In [44]:
lenet_loaded_model = tf.keras.models.load_model("lenet")
lenet_loaded_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 220, 220, 6)       456       
                                                                 
 batch_normalization_6 (Bat  (None, 220, 220, 6)       24        
 chNormalization)                                                
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 110, 110, 6)       0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 106, 106, 16)      2416      
                                                                 
 batch_normalization_7 (Bat  (None, 106, 106, 16)      64        
 chNormalization)                                                
                                                        

In [46]:
LenetModel.save("lenet.hdf5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [47]:
lenet_loaded_model = tf.keras.models.load_model("lenet.hdf5")
lenet_loaded_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 220, 220, 6)       456       
                                                                 
 batch_normalization_6 (Bat  (None, 220, 220, 6)       24        
 chNormalization)                                                
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 110, 110, 6)       0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 106, 106, 16)      2416      
                                                                 
 batch_normalization_7 (Bat  (None, 106, 106, 16)      64        
 chNormalization)                                                
                                                        

In [50]:
lenet_loaded_model.evaluate(test_dataset)

87/87 [==============================] - 18s 53ms/step - loss: 0.1747 - accuracy: 0.9427


[0.17473694682121277, 0.9426705241203308]

In [48]:
lenet_model.save_weights("weights/lenet_weights")

In [49]:
lenet_weights_model = lenet_model.load_weights("weights/lenet_weights")